In [265]:
from IPython.display import HTML
display(HTML("<head><link rel='stylesheet' type='text/css' href='./../../static/custom.css'></head>"))
display(HTML("<style>.container { width:100% !important; }</style>"))

In [266]:
import numpy as np

from bqplot import *
import bqplot as bq
import bqplot.marks as bqm
import bqplot.scales as bqs
import bqplot.axes as bqa

import ipywidgets as widgets

In [267]:
def get_n(z, dens, mass_mol):
    n = z*Na * dens/mass_mol
    return n

In [268]:
def get_Ef(n):
    Ef = h**2/(2*me)*(3*pi**2*n)**(2.0/3.0)
    return Ef

In [269]:
def get_mu(Ef,T):
    mu = Ef*(1.0-1.0/3.0*(pi*kb*T/2.0/Ef)**2)
    return mu

In [270]:
def get_g(eps,Ef,n):
    g = 3.0/2.0*n/Ef * np.sqrt(eps/Ef)
    return g

In [271]:
def calculate_integrals(eps_values, y_values):
    
    n_of_elec = 0.0
    U_total = 0.0
    y_values = y_values
    de = (eps_values[pts-1] - eps_values[0])/pts
    for i in range(pts):
        n_of_elec = n_of_elec + y_values[i]*de
        U_total = U_total + y_values[i]*eps_values[i]*de
    return n_of_elec, U_total

In [272]:
def update_temp(change):    
    T = T_slider.value
    Ef = float(fermi_energy_text.value)
    mu = get_mu(Ef,T)
    
    mu_plot_x = np.linspace(mu,mu,pts)
    mu_plot.x = mu_plot_x

  
    n_values = np.empty(pts)
    g_values = np.empty(pts)
    y_values = np.empty(pts)
    for i in range(pts):
        eps = eps_values[i]
        n_values[i] = 1.0/(e**((eps-mu)/kb/T)+1.0)
        g_values[i] = get_g(eps_values[i], Ef, n)
        y_values[i] = n_values[i] * g_values[i]
    curve.y = y_values
    lines_001.y = g_values
    lines_002.y = n_values
    
    mu_text.value ='%.2f' % mu
    n_of_elec, U_total = calculate_integrals(eps_values, y_values)
    u_average = U_total/n_of_elec
    u_reduced = u_average/Ef
    
    u_average_text.value = '%.2f' % u_average
    u_reduced_text.value = '%.2f' % u_reduced

In [273]:
def update_metal(change):
    metal = metal_dropdown.value
    rho = metal_densities[metal]
    mass = molar_masses[metal]
    z = valence_numbers[metal]
    n = get_n(z, rho, mass)
    Ef = get_Ef(n)
    Tf = Ef/kb
       
    fermi_energy_text.value = '%.2f' % Ef
    fermi_temp_text.value = '%.2f' % Tf
    
    update_temp(None)

In [274]:
pts = 2000
dim = 3
T = 2.0
T_min = 0.1
T_max = 10.0
h = 6.5821e-16 #Planck's reduced constant in eV*s
c = 3.0e8 # Speed of Light in m/s
me = 510998.9/c**2 # Electron mass in eV/c**2
Na = 6.0221e23
kb = 8.617e-1 # Boltzmann's in eV/(10**4K)
pi =  np.pi
e = np.e

# Metal data:
metal_labels = ["Sodioa", "Aluminioa", "Burdina", "Kobrea"] 
metal_densities = [968.0, 2700.0, 7874.0, 8960.0] # In kg/m**3
molar_masses = [0.0229, 0.026, 0.0558, 0.0635] # In Kg
valence_numbers = [1, 3, 2, 1]

# Test calculation
metal = 1
rho = metal_densities[metal]
mass = molar_masses[metal]
z = valence_numbers[metal]
n = get_n(z, rho, mass)
Ef = get_Ef(n)
Tf = Ef/kb
mu = get_mu(Ef,T)



eps_values = np.linspace(0.0,20.0,pts)
g_values = np.empty(pts)
for i in range(pts):
    g_values[i] = get_g(eps_values[i], Ef, n)
    
    
n_values = np.empty(pts)    
y_values = np.empty(pts)
for i in range(pts):
    eps = eps_values[i]
    n_values[i] = 1.0/(e**((eps-mu)/kb/T)+1.0)
    y_values[i] = n_values[i] * g_values[i]
mu_plot_x = np.linspace(mu,mu,pts)
mu_plot_y = np.linspace(0.0,1.0,pts)

In [275]:
# Figures
fig_133_001 = bq.Figure(title='Egoera-dentsitatea',
                marks=[],
                axes=[],
                animation_duration=0,
                legend_location='top-right',
                legend_style= {'fill': 'white', 'stroke': 'grey'},
                background_style= {'fill': 'white',  'stroke': 'black'},
                fig_margin=dict(top=70, bottom=60, left=80, right=30),
                layout = widgets.Layout(width='50%'),
                toolbar = True,
    )

fig_133_002 = bq.Figure(title='Fermi-Dirac banaketa',
                marks=[],
                axes=[],
                animation_duration=0,
                legend_location='top-right',
                legend_style= {'fill': 'white', 'stroke': 'grey'},
                background_style= {'fill': 'white',  'stroke': 'black'},
                fig_margin=dict(top=70, bottom=60, left=80, right=30),
                layout = widgets.Layout(width='50%'),
                toolbar = True,
    )

scale_x_001 = bqs.LinearScale(min = np.min(eps_values), max = np.max(eps_values))
scale_y_001 = bqs.LinearScale(min = 0.0, max = 5.0e28)

scale_x_002 = bqs.LinearScale(min = np.min(eps_values), max = np.max(eps_values))
scale_y_002 = bqs.LinearScale(min = 0.0, max = 1.0)

axis_x_001 = bqa.Axis(scale=scale_x_001,
                tick_format='.0f',#'0.2f',
                tick_style={'font-size': '15px'},
                #tick_values = np.linspace(p_min, p_max, 7),
                num_ticks=3,
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                label='e (eV)',
                label_location='middle',
                label_style={'stroke': 'black', 'default-size': 35},
                label_offset='50px')

axis_y_001 = bqa.Axis(
                scale=scale_y_001,
                tick_format='1.3E',#'0.2f',
                tick_style={'font-size': '15px'},
                tick_values=np.linspace(0.0,5.0e28,6),
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                orientation='vertical',
                label='g (eV**-1 m**-3)',
                label_location='middle',
                label_style={'stroke': 'red', 'default_size': 35},
                label_offset='60px')

axis_x_002 = bqa.Axis(scale=scale_x_002,
                tick_format='.0f',#'0.2f',
                tick_style={'font-size': '15px'},
                #tick_values = np.linspace(p_min, p_max, 7),
                num_ticks=3,
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                label='e (eV)',
                label_location='middle',
                label_style={'stroke': 'black', 'default-size': 35},
                label_offset='50px')

axis_y_002 = bqa.Axis(
                scale=scale_y_002,
                tick_format='.1f',#'0.2f',
                tick_style={'font-size': '15px'},
                num_ticks=6,
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                orientation='vertical',
                label='ni',
                label_location='middle',
                label_style={'stroke': 'red', 'default_size': 35},
                label_offset='50px')


fig_133_001.axes = [axis_x_001, axis_y_001]
fig_133_002.axes = [axis_x_002, axis_y_002]


lines_001 = bqm.Lines(
            x = eps_values, 
            y = g_values, 
            scales = {'x': scale_x_001, 'y': scale_y_001}, 
            opacities = [1.0],
            visible = True, #True, #t == '1.00',
            colors = ['green'],
            labels = ['Egoera-dentsitatea'],
            display_legend = False
)

curve = bqm.Lines(
            x = eps_values, 
            y = y_values, 
            scales = {'x': scale_x_001, 'y': scale_y_001}, 
            opacities = [1.0],
            visible = True, #True, #t == '1.00',
            colors = ['red'],
            labels = ['Elektroi kopurua'],
            fill = 'bottom',
            fill_opacities = [0.2],
            display_legend = False,
)


lines_002 = bqm.Lines(
            x = eps_values, 
            y = n_values, 
            scales = {'x': scale_x_002, 'y': scale_y_002}, 
            opacities = [1.0],
            visible = True, #True, #t == '1.00',
            colors = ['blue'],
            labels = ['Egoera-okupazioa'],
            display_legend = False
)

mu_plot = bqm.Lines(
            x = mu_plot_x, 
            y = mu_plot_y, 
            scales = {'x': scale_x_002, 'y': scale_y_002}, 
            opacities = [0.4],
            visible = True, #True, #t == '1.00',
            colors = ['orange'],
            labels = ['Fermi maila'],
            display_legend = False
)

fig_133_001.marks = [lines_001, curve]
fig_133_002.marks = [lines_002, mu_plot]


#Widgets

T_slider = widgets.FloatSlider(
    min=T_min,
    max=T_max,
    step=(T_max-T_min)/pts,
    value=T,
    description='T',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
    layout = widgets.Layout(width='90%'),
)

T_slider.observe(update_temp, 'value')

metal_dropdown = widgets.Dropdown(
    options=[('Sodioa', 0), ('Aluminioa', 1), ('Burdina', 2), ('Kobrea', 3)],
    value=metal,
    description='Gasa:',
    disabled=False,
)

metal_dropdown.observe(update_metal, 'value') 


# Outputs

fermi_energy_text = widgets.Label(value='%.2f' % Ef)
fermi_temp_text = widgets.Label(value='%.2f' % Tf)
mu_text = widgets.Label(value='%.2f' % mu)
u_average_text = widgets.Label(value='%.2f' % 0.0)
u_reduced_text = widgets.Label(value='%.2f' % 0.0)



# Layout

left_block_133_000 = widgets.VBox([], layout=widgets.Layout(width='30%', align_self='center', align_items='center'))
left_block_133_000.children = [metal_dropdown,
                               widgets.HBox([widgets.Label(value='$\epsilon_F=$'),fermi_energy_text,widgets.Label(value='$eV$')]),
                               widgets.HBox([widgets.Label(value='$T_F=$'),fermi_temp_text,widgets.Label(value='$x10^4 K$')]),
                               widgets.HBox([T_slider, widgets.Label(value='$x10^4 K$')], layout=widgets.Layout(width='100%')),
                               widgets.HBox([widgets.Label(value='$\mu=$'),mu_text,widgets.Label(value='$eV$')]),
                               widgets.HBox([widgets.Label(value='$<U>=$'),u_average_text,widgets.Label(value='$eV$')]),
                               widgets.HBox([widgets.Label(value='$<U>$'),u_reduced_text,widgets.Label(value='$\epsilon_F$')])
                              ]

center_block_133_000 = widgets.HBox([], layout=widgets.Layout(width='70%', align_self='center', align_items='center'))
center_block_133_000.children = [fig_133_002, fig_133_001]

#right_block_133_000 = widgets.VBox([], layout=widgets.Layout(width='20%', align_self='center', align_items='center'))
#right_block_133_000.children = [T_slider]

main_block_133_000 = widgets.HBox([], layout=widgets.Layout(width='100%', align_self='center', align_items='center'))
main_block_133_000.children = [left_block_133_000, center_block_133_000]

main_block_133_000